# BT Analysis
The goal here is to:

-   Extract the full lengths cds
    -   Applying this to both the Crickmore database as well as the Patric data seems to make the most sense going forward - things will just be easier.
-   Check to see if there Patric sequences are full length CDS when compared to
    the Crickmore database. This will be done using blast at the nt/prot level.
    -   If yes then proceed as normal.
    -   If no then try to build out the assembly for each contig.
-   Create a tree showing divergence with the nucleotide sequences of each
    Patric cry protein amongst those registered in the Crickmore db.
-   Analyze the protein domains
    -   Need more details about this analysis

## Get Full Length CDS

### Crickmore
To obtain known cry sequences I used the Crickmore database to grab the accessions for each gene by parsing the [html file](http://www.btnomenclature.info/). I provided all of these to [batch entrez](https://www.ncbi.nlm.nih.gov/sites/batchentrez) and downloaded the protein which were available. For those that were not found in the protein database, I provided those accessions to the nucleotide database and grabbed the CDS. Of all the crickmore sequences only 12 were not found:

The following records can't be retrieved:

- Id=KJ28844:	nuccore: Wrong UID KJ28844
- Id=KJ28845:	nuccore: Wrong UID KJ28845
- Id=KJ28846:	nuccore: Wrong UID KJ28846
- Id=MG674828:	nuccore: Wrong UID MG674828
- Id=MG983752:	nuccore: Wrong UID MG983752
- Id=MG983753:	nuccore: Wrong UID MG983753
- Id=MG983754:	nuccore: Wrong UID MG983754
- Id=MH253686:	nuccore: Wrong UID MH253686
- Id=MH475904:	nuccore: Wrong UID MH475904
- Id=MH475905:	nuccore: Wrong UID MH475905
- Id=MH475906:	nuccore: Wrong UID MH475906
- Id=MH475907:	nuccore: Wrong UID MH475907

Also I manually omitted AL731825 and CP015350 from `cry-accessions.txt` as they were improperly annotated in NCBI.

In [3]:
import os
os.chdir('../crickmore/cry')

In [19]:
## Commented out because I do not need to run them again.

# %%bash
# reformat fasta files to 1 line fasta to allow grep to work easier
# perl ../../../1line-fasta.pl crickmore-cds.fasta > crickmore-cds.1line.fasta
# grep --no-group-separator -A1 -F -f cry-accessions.txt crickmore-cds.1line.fasta  > temp && mv temp crickmore-cds.fasta

Since there were some NCBI entries that were the CDS I translated these into proteins using [transeq](https://www.ebi.ac.uk/seqdb/confluence/display/THD/EMBOSS+Transeq) from the [EMBOSS sotware suite](https://www.ebi.ac.uk/Tools/emboss/).

In [ ]:
## Commented out because I do not need to run them again.

# %%bash
# transeq -clean -sequence crickmore-cds.fasta -outseq crickmore-cds.translated.fasta -frame 1

# Combine them with the existing protein sequences
# cat crickmore-aa.fasta crickmore-cds.translated.fasta > all-cry-aa.fasta

### PATRIC

To start I wanted to see how well blastx matched up with the ToxinScanner results as I will be using blastx (and other flavors) for future analysis and want to make sure that I have the parameters correct. For a simple test I ran it on the d12 sample.

In [4]:
os.chdir('../../patric/d12')

In [ ]:
%%bash

blastx \
-query a1-d12.contigs.fasta \
-db all-cry-aa.fasta \
-out contig-blastx.tsv \
-outfmt '6 qaccver saccver pident length qlen slen mismatch gapopen qstart qend sstart send evalue bitscore' \
-num_threads 4 \
-max_target_seqs 1 \
-culling_limit 1 \
-evalue 0.00005 \
-max_hsps 1

In [7]:
import pandas as pd
import tabulate
df = pd.read_table('contig-blastx.tsv',header=None)
df.columns = 'qaccver saccver pident length qlen slen mismatch gapopen qstart qend sstart send evalue bitscore'.strip().split(' ')
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.precision', 2):
        print(tabulate.tabulate(df, headers='keys', tablefmt='markdown'))

    qaccver                 saccver      pident    length    qlen    slen    mismatch    gapopen    qstart    qend    sstart    send    evalue    bitscore
--  ----------------------  ---------  --------  --------  ------  ------  ----------  ---------  --------  ------  --------  ------  --------  ----------
 0  accn|91061.11.con.0048  Cry6Ba1      30.571       350   17299     395         217          6      8651    9622        43     392  2.25e-32       129
 1  accn|91061.11.con.0049  Cry55Aa3     26.855       283   17477     363         180          8      3098    2307        68     342  2.78e-12        68.2
 2  accn|91061.11.con.0054  Cry6Aa1      28.736       348   15326     475         224          5      6907    7878        37     384  8.96e-37       144
 3  accn|91061.11.con.0055  Cry75Aa3     33.784       222   15041     317         128          8      5681    5064        94     312  3.7e-18         85.1
 4  accn|91061.11.con.0060  Cry4Ba2      76.577       111   12780    1136 

So the results matched the ToxinScanner results for the d12 sample. However, this will need a but more tweaking as some of the alignments (in the `length` field) are very short. To summarize:
* All potential coding sequences from the alignment coordinates (and beyond so that we don't lose miss any potential start/stop codons that are in frame) within the matched contig will be extracted
* Those coding sequences that align to one of our putative toxins will be considered as potential candidates.
* Of these candidates, we will look for domains (?) and/or grab the longest CDS and consider that our toxin.

The initial attempt of this can be seen below in a latter section (coming soon!).

## Alignment
In order to see how divergent our sequences are from the known Cry proteins we will need to construct a multiple-sequence alignment from which we can produce a tree. Since both the Crickmore database as well as the predicted genes from ToxinScanner are not solely full length CDS I had to decide on some cutoffs until we are certain that the data we are working with is perfect. Also, it takes ~8 hours to generate a tree for the 438 sequences that we're working with now - so given time constraints I decided these cutoffs were best for a proof-of-concept. 

A few questions I have about the alignment/tree generation steps are as follows:
- I have read that for some distance based methods the alignment does not have to be perfect and can actually afford many gaps but would like to know for sure.
- There are many methods to estimate distances between sequences. The crockmore database used UPGMA/neighbor-joining approaches to calculate distances which is good for sequences that are fairly similar. I used a maximum likelihood approach which is tailored to sequences that are very dissimilar. Given that our sequences can be <= 40% identical (to me that seems low) this seemed the best approach to me.
- The Crickmore team used a rooted tree which assumes there is a common ancestor. As far as I know that isn't the case here, so I initially generated an unrooted tree which is better for looking at variability amongst genes as opposed to evolution over time. This is also something I would like more clarity on.

### Filtering Sequences
Given the constraints of the initial analysis, I took the libery of paring down the data into something that I'm more confident in working with as well as something that is quicker. 

From the PATRIC data files I retrieved each of the strain's predicted protein sequences mainly due to the fact that it was a bit of a rabbit hole in trying to find the full lenght CDS based on partial alignments.

These protein sequences were then blasted against the Crickmore datbase. From the blast results I filter the hit for the sequences that:
- have a minimum alignment length of 100aa
- have a minumum length of 1kaa
- have an alignment that covers at least 80% of the sequence of interest
- Have a start and stop codon/aa

All sequences that passed these filtered were then passed on to the alignment algorithm.

In [1]:
os.chdir('..')

a24/                                          BT_Analysis.ipynb  j1/
all-patric-and-crickmore.aa.1k.fasta          d12/               mb11/
all-patric-and-crickmore.aa.1k.mat            font1@             my46/
all-patric-and-crickmore.aa.1k.phy            font2@             plotfile
all-patric-and-crickmore.aa.1k.phy.out        font3@             plotfile.pdf
all-patric-and-crickmore.aa.1k.phy.tree       font4@             s1/
all-patric-and-crickmore.aa.1k.phy.tree.orig  font5@
analysis.sh@                                  font6@


In [3]:
%%bash
for i in a24 j1 my46 mb11 s1 d12
 do
  blastp -query  $i/PATRIC_genome_feature.fasta -db ../crickmore/cry/all-cry-aa.fasta -out $i/PATRIC_genome_feature.blastp.tsv -outfmt '6 qaccver saccver pident length qlen slen mismatch gapopen qstart qend sstart send evalue bitscore' -num_threads 4 -max_target_seqs 3 -max_hsps 2 -culling_limit 2 -evalue 0.00005

  python3 ../../scripts/filter-blast-results.py -i $i/PATRIC_genome_feature.blastp.tsv  -l 100 -q 80 --qlen 1000 -s $i/PATRIC_genome_feature.fasta -o $i/PATRIC_genome_feature.blastp.filtered.fasta

done

### Alignment

For the alignment algorithm I used clustal omega.

In [ ]:
%%bash

# Filtered the Crickmore database for seqs >= 1kaa
python ../../scripts/filter-seqs.py -s ../crickmore/cry/all-cry-aa.fasta -l 1000 --full > ../crickmore/cry/all-cry-aa.1k.fasta

# Concatenate all the Crickmore and sequences of interest into one file
cat ../crickmore/cry/all-cry-aa.1k.fasta */PATRIC_genome_feature.blastp.filtered.fasta > all-patric-and-crickmore.aa.1k.fasta

# Align using clustal and outputting phylip format
clustalo --outfmt phy --distmat-out all-patric-and-crickmore.aa.1k.mat -v -i all-patric-and-crickmore.aa.1k.fasta -o all-patric-and-crickmore.aa.1k.phy --force --full

### Distance Matrix Calculation
To generate a distance matrix I used [proml](http://evolution.genetics.washington.edu/phylip/doc/proml.html) from the PHYLIP package. Proml is an algorithm that implements maximum liklihood approach to generating a distance matrix. The Crickmore people used [protdist](http://evolution.genetics.washington.edu/phylip/doc/protdist.html) which, as I stated earlier, didn't seem like the right tool for this job. 

proml is an interactive piece of software so I do not have the commands listed here. However, I used the default parameters.

FYI this is a very lengthy algorithm (~8 hours for 438 sequences).

### Tree Drawing

I used [FigTree](http://tree.bio.ed.ac.uk/software/figtree/) to render the tree.

## Cluster Analysis
A way to see which of our genes are considered 'novel' is to look at how they cluster when given a certain identity threshold. To do this I used CD-HIT to cluster based on percent identity between sequences. Since there are different percent identities that segregate the different Cry ranks I will use those to start.

In [4]:
%%bash
for i in 45 78 95
do 
cd-hit \
-i all-patric-and-crickmore.aa.1k.fasta \
-o all-patric-and-crickmore.aa.1k.$i.cdhit \
-T 4 -g 1 -c .$i
done

Program: CD-HIT, V4.7 (+OpenMP), Jul 13 2018, 17:17:44
Command: cd-hit -i all-patric-and-crickmore.aa.1k.fasta -o
         all-patric-and-crickmore.aa.1k.45.cdhit -T 4 -g 1 -c
         .45

Started: Thu Oct 11 03:37:20 2018
                            Output                              
----------------------------------------------------------------
Program: CD-HIT, V4.7 (+OpenMP), Jul 13 2018, 17:17:44
Command: cd-hit -i all-patric-and-crickmore.aa.1k.fasta -o
         all-patric-and-crickmore.aa.1k.78.cdhit -T 4 -g 1 -c
         .78

Started: Thu Oct 11 03:37:20 2018
                            Output                              
----------------------------------------------------------------
total seq: 402
longest and shortest : 1523 and 1044
Total letters: 477041
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 4 X 10M = 43M
Table           : 2 X 65M = 130M
Miscellaneous   : 0M
Total           : 174M

Table limit with t


Fatal Error:
Too low cluster threshold for the word length.
Increase the threshold or the tolerance, or decrease the word length.
Program halted !!



It seems that 45% is too low an identity threshold for that word length. However, at 95% there are 178 clusters and at 78% there are 119 clusters. I will have to look to see where the others fall into this category.